In [196]:
%matplotlib notebook
import json
import matplotlib.pyplot as plt
import pandas as pd
import requests
import pprint
import time
import datetime
import csv
import calendar

In [2]:
url = "https://app.pendo.io/api/v1/report"
headers = {
    'x-pendo-integration-key': "713bb559-8e38-47bd-51e5-911685ce9113",
    'content-type': "application/json"
}

accurl = "https://app.pendo.io/api/v1/account/105672"

response = requests.get(accurl, headers = headers)

In [3]:
response.json()

{'id': '105672',
 'metadata': {'agent': {'accountlevel': 12,
   'accounttype': 0,
   'ampseaenabled': False,
   'campaignalignmentcustomer': 0,
   'country': 'United States',
   'created': 1438755390000,
   'currency': 0,
   'engagecustomer': True,
   'shard': 67,
   'timezone': 'America/Los_Angeles',
   'wavecustomer': False},
  'auto': {'firstvisit': 1466536450257,
   'id': '105672',
   'idhash': 1611050280,
   'lastupdated': 1562363703704,
   'lastvisit': 1562363657920,
   'nid': 105672},
  'auto__323232': {'firstvisit': 1534354797682, 'lastvisit': 1562363657920},
  'salesforce': {'pardot_account_id__c': 105672}}}

## Psuedo Code Ideas 
The purpose of the effort is to use usage data to calculate real time-to-value for our existing customers. We define value as sending the first email. So the research question is, on average, how long from the time a user is first created in Pardot,  does it take for them to send the first email? 

Now I have the ability to pull daily visitor history for each visitor over the past 6 months. So if I have a cohort of users whose day 1s were all on Feb 1 I would want to capture what date each user sent their 1st email. 

#### Objective 1: User & Date of 1st Email Send Table
So TABLE ! should be User / Date of first email send. then I would want to capture the number of days that passed between the send of the first email and their first day in the app. 

#### Objective 2: New Column with # of days from 1st send to creation
So TABLE 1.5 would have an appended column with the # of days (or hours..whatever unit of time) between the emaiul send date, and their created date feb 2. 

#### Objective 3: Average
Then I'd want an average. So I would calculate average of the 3rd column in Table 1.5 to say what the average time to value is. 

This is the simplest output. Ideally what I would like, is to identify the average time to reach multiple critical touchpoints in the setup journey. For instance how long does it take to import prospects, setup folders, create first templates create users. 

Additionally I would like to be able to discuss differences by segment. What are the differences by company size? What abour our MBUs companies? Our ESMBs (for essentials) 

## CHALLENGES
1) The amount of data is so huge! For each person, each day produces a ton of json objects describing activity... most of which I'm not concerned with. What is the most efficient way to drop everything except for milestone moments? 

2) How do I iteratively tell it to pull data for the users in this particular cohort? A loop...but then feels like I'll have to also do some sort of export 

3) Gotta look into the time translations. 

4) Defining the starting point.....How do we use people who start? Msybe look at Org 62? 

5) When did they buy it? Org62? 


## OBJECTIVE 1A: TABLE WITH EMAIL SEND DATES FOR 1 VISITOR

In [4]:


# Extract Visitor 19721543's History for a Time period 5/8/2019
vurl = "https://app.pendo.io/api/v1/visitor/13038201/history"
params = {
    "starttime": "1560142800000"
}
headers = {
    'x-pendo-integration-key': "713bb559-8e38-47bd-51e5-911685ce9113",
    'content-type': "application/x-www-form-urlencoded"
}
response = requests.get(vurl, headers = headers, params = params)

In [5]:
#Print visitor's data for the day
data =response.json()
#print(json.dumps(data, indent=4, sort_keys=True))
data

[{'type': 'page',
  'uri': '://app.pendo.io/api/s/6123269936644096/page/gSuMnhiQKbtCyya8mzQIwQLK-8k',
  'parsedUserAgent': {'name': 'Chrome',
   'version': '74.0.3729',
   'os': 'Windows'},
  'appId': -323232,
  'pageId': 'gSuMnhiQKbtCyya8mzQIwQLK-8k',
  'ts': 1560178680000,
  'lastTs': 1560178731896,
  'duration': 51896},
 {'type': 'page',
  'uri': '://app.pendo.io/api/s/6123269936644096/page/s6Y_-4Tx5zwtILqlJLxuGPr4cEI',
  'parsedUserAgent': {'name': 'Chrome',
   'version': '74.0.3729',
   'os': 'Windows'},
  'appId': -323232,
  'pageId': 's6Y_-4Tx5zwtILqlJLxuGPr4cEI',
  'ts': 1560178680000,
  'lastTs': 1560178731896,
  'duration': 51896},
 {'type': 'page',
  'uri': '://app.pendo.io/api/s/6123269936644096/page/txS7C31PH6w3dLoPppolH61daBg',
  'parsedUserAgent': {'name': 'Chrome',
   'version': '74.0.3729',
   'os': 'Windows'},
  'appId': -323232,
  'pageId': 'txS7C31PH6w3dLoPppolH61daBg',
  'ts': 1560178680000,
  'lastTs': 1560178731896,
  'duration': 51896},
 {'type': 'page',
  'uri'

In [6]:
length=len(data)
#with open('data2.txt', 'w') as outfile:  
    #json.dump(data, outfile)

In [7]:
#If a user has a featureid for email send, add to a list with the date the feature was fired
emailsend_id = "t0e5eNJp9X61tFyqt7eeXv0mEPo"
    
#I want the program to loop through each list item in data (increment) and if the featureid matches the send email id, add to a list
#setup list to hold the send email ID
feature_List= []
Date = []
UserID= []

for pid in range(length): 
    if data[pid]["type"]== "feature" and data[pid]["featureId"]== emailsend_id:
        Feature = data[pid]["featureId"]
        StartTime = data[pid]["ts"]
        feature_List.append(Feature)
        Date.append(StartTime)
        continue


#NEXT,FOCUS ON DYNAMICALLY POPULATING DIFFERENT USERS AND DIFFERENT DATES.       
#https://app.pendo.io/accountlist/LVBTGHAfad1en534nDACn8IHcwQ import csv
print(Date)


[1560179167888, 1560179224169, 1560179277863, 1560179328612, 1560179379678, 1560193837030, 1560193912358, 1560193980568]


## Objective 1B: USE NESTED FOR LOOPS TO LOOP THROUGH A SMALL SET OF USERS AND SMALL SET OF DATES


In [237]:
#Setup test inputs for the loops
baseurl="https://app.pendo.io/api/v1/visitor/"
Users = ["15073483", "15560481", "15825351"]
Time = ["1560398400000","1560484800000","1560744000000", ]
Send_date=[]
df_Send_date=[]
URLs = []
df_user = []


#Loop 1: Build the base URL and pull values from a list of user IDs 
for user in cohort_list:
    queryurl= baseurl+ user+ "/history" 
    #Loop 2: Complete the API URL and add the date to paramaters from a list. Then look for a feature match with sending an email. 
    for date in epoch_list: 
        params =  params = {"starttime": date}   
        headers = {'x-pendo-integration-key': "713bb559-8e38-47bd-51e5-911685ce9113",
   'content-type': "application/x-www-form-urlencoded"}
        response = requests.get(queryurl, headers = headers, params = params)
        data = response.json()
        Length=len(data) 
        for pid in range(Length): 
            if data[pid]["type"]== "feature" and data[pid]["featureId"]== emailsend_id:
                Feature = data[pid]["featureId"]
                StartTime = data[pid]["ts"]
                feature_List.append(Feature)
                Send_date.append(StartTime)
                df_user.append(user)
                break
            else:
                continue
            break
        
        


KeyboardInterrupt: 

In [226]:
#change format of the dates so they are readeablee
#for date in Send_date: 
    #a= time.gmtime(date /1000.)
    #new_date= time.strftime('%m/%d/%Y %H:%M:%S', a)
    #df_Send_date.append(new_date)

print(Send_date)
print(df_user)



[1560782624622, 1560538449814, 1560802517501, 1560426271742, 1560781791591]
['15073483', '15560481', '15560481', '15825351', '15825351']


In [227]:
#Create a dataframe from lists
Send_Email_DF = pd.DataFrame()
Send_Email_DF['UserID'] = df_user
Send_Email_DF['Email_Send_Date']= Send_date

#Send_Email_DF = Send_Email_DF[Send_Email_DF.Email_Send_Date != "0"]
Send_Email_DF
#time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(a))



,UserID,Email_Send_Date
0,15073483,1560782624622
1,15560481,1560538449814
2,15560481,1560802517501
3,15825351,1560426271742
4,15825351,1560781791591


In [235]:
#RESET AS NEEDED
df_Send_date.clear()
Send_date.clear()
URLs.clear()
df_user.clear()
human_dates.clear()

print(df_Send_date)
print(Send_date)
print(df_user)
print(human_dates)
print(epoch_list)

[]
[]
[]
[]
['1552435200000', '1552521600000', '1552608000000', '1552694400000', '1552780800000', '1552867200000', '1552953600000', '1553040000000', '1553126400000', '1553212800000', '1553299200000', '1553385600000', '1553472000000', '1553558400000', '1553644800000', '1553731200000', '1553817600000', '1553904000000', '1553990400000', '1554076800000', '1554163200000', '1554249600000', '1554336000000', '1554422400000', '1554508800000', '1554595200000', '1554681600000', '1554768000000', '1554854400000', '1554940800000', '1555027200000']


In [228]:
#Convert epochs to human readable dates 
human_dates=[]
for epoch in Send_date: 
    s = epoch / 1000.0
    human= datetime.datetime.fromtimestamp(s).strftime('%Y-%m-%d %H:%M:%S.%f')
    human2 = human.split(" ", 1)[0]
    human_dates.append(human2)

Send_Email_DF['Readable_Date']= human_dates
Send_Email_DF.sort_values(["UserID"], axis=0, 
                 ascending=False, inplace=False) 
Send_Email_DF

,UserID,Email_Send_Date,Readable_Date
0,15073483,1560782624622,2019-06-17
1,15560481,1560538449814,2019-06-14
2,15560481,1560802517501,2019-06-17
3,15825351,1560426271742,2019-06-13
4,15825351,1560781791591,2019-06-17


In [216]:
#calculate day delta and add to dataframe 
deltas=[]

for x in human_dates: 
    d0 = datetime.datetime.strptime(x, '%Y-%m-%d').date()
    d1= datetime.datetime.strptime('2019-03-13', '%Y-%m-%d').date() 
    delta = d0-d1
    deltas.append(delta.days)
    

Send_Email_DF['# of days']= deltas
Send_Email_DF


,UserID,Email_Send_Date,Readable_Date,# of days
0,15073483,1560782624622,2019-06-17,96
1,15560481,1560538449814,2019-06-14,93
2,15560481,1560802517501,2019-06-17,96
3,15825351,1560426271742,2019-06-13,92
4,15825351,1560781791591,2019-06-17,96


### You have the bones. Next steps 

1) Now you need to figure out how to get the list of specific cohort user values who all started the same date

2) Designate that date as the static url 

3) Figure out how to loop through every day from the start date through a period of time to pull the values of email sends. 

4) Run those values through existing code to get the dataframe

5) turn the #of days column into integers and calculate the average


In [236]:
from datetime import date, datetime, timedelta


#import list of users in the 3.11 cohort 
df = pd.read_csv('3.11.cohort.csv')
cohort_list = df['Visitor ID'].tolist()
cohort_list = [str(i) for i in cohort_list] 



epoch_list=[]
human_list=[]
#designate start date
#start_date =datetime.datetime.strptime('2019-03-11', '%Y-%m-%d').date() 

#List of epoch dates from 3.11 - 4.11 
def datelist (start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
        
for dates in datelist(date(2019, 3, 13), date(2019, 3, 20), timedelta(days=1)):
    dates = datetime.strftime(dates, "%Y, %m, %d")
    human_list.append(dates)
    
#change dates to epoch format in milliseconds
for date in human_list: 
    epoch = calendar.timegm(time.strptime(date, '%Y, %m, %d'))
    milli_epoch=epoch * 1000
    milli_epoch = str(milli_epoch)
    epoch_list.append(milli_epoch)

epoch_list    

['1552435200000',
 '1552521600000',
 '1552608000000',
 '1552694400000',
 '1552780800000',
 '1552867200000',
 '1552953600000']

In [162]:
cohort_list.clear()
#human_list.clear()
#epoch_list.clear()

In [160]:
type(epoch_list[1])

str